# Runfiles for NonLinLoc: Steps Vel2Grid + Grid2Time + NLLoc

This notebook was made to modify templates of runfiles for the NonLinLoc software. 

Firstly, for the grid files you can use the Matching_VelModel notebook to create 2 .txt files that will be needed to modify the template of the runfile. It's made to add the lines for 2 commands: VGGRID and LAYER. <br> Secondly, it modifies the command lines for the runfile made for location of events with NLLoc. All the parameters needed can be entered here and the modifications will be made on the template.

In [1]:
base_dir = '/Users/lpapin/Documents/phd/projects/sw4/nonlinloc/'

For now everything is in local but since NonLinLoc is on the cluster, let's try to keep a similar organization than the creator:

- NLLoc as the main folder
- NLLoc/model for the model grid
- NLLoc/time for the travel-time grid
- NLLoc/obs for the picks
- NLLoc/run for the runfiles
- NLLoc/loc for the locations

NB: #LP is added to each line modified thanks to this notebook to remember which lines were the original ones.

# 1. Travel-time grids

Vel2Grid and Vel2Grid3D have slightly different input. They have common commands but the first one use LAYER while the second one has VGINP and VGCLIP. 

In [2]:
# Precise here which one is used to run the notebook entirely
which = 'vel2grid3d' # 'vel2grid

## Vel2Grid

*Given a velocity model description, Vel2Grid generates a 3D model Grid header and buffer files containing velocities, slownesses or other model specification.*

First we're modifying the line giving the grid description VGGRID for Vel2Grid.

In [3]:
# Read the original .run file
with open(base_dir+'ttgrid_runfile_template.in', 'r') as file:
    lines = file.readlines()

# Read the second line directly from VGGRID_command.txt
with open(base_dir+'VGGRID_command.txt', 'r') as file:
    lines_to_add = file.readlines()
    vggrid_command = lines_to_add[1].strip() 

# Find and replace the VGGRID line in the .run file
for i, line in enumerate(lines):
    if line.startswith('VGGRID'):
        lines[i] = vggrid_command + ' #LP' + '\n'
        break

# Write the modified content back to a new file (or overwrite the original)
with open(base_dir+'ttgrid_runfile_final.in', 'w') as file:
    file.writelines(line.rstrip() + '\n' for line in lines)

print('VGGRID line has been modified successfully.')

VGGRID line has been modified successfully.


Now we add the lines giving the velocity model layers. It is used for a type of model where the velocites don't change with lat and lon (hence the correction for the SVI project).

In [4]:
if which=='vel2grid':
    # Read the second line directly from LAYER_command.txt
    with open(base_dir+'LAYER_command.txt', 'r') as file:
        layer_lines = [line.strip() for line in file.readlines()[2:]]
    layer_lines = [f'{line} #LP' for line in layer_lines]
    
    # Find the line to insert the LAYER commands in the .run file
    for i, line in enumerate(lines):
        if line.startswith('#LAYER depth Vp_top Vp_grad Vs_top Vs_grad p_top p_grad'):
            insertion_point = i + 1  # Insert after this line
            break
    
    # Insert the LAYER lines at the specified point
    modified_lines = lines[:insertion_point] + layer_lines + lines[insertion_point:]
    
    # Write the modified content back to a new file (or overwrite the original)
    with open(base_dir+'ttgrid_runfile_final.in', 'w') as file:
        file.writelines(line.rstrip() + '\n' for line in modified_lines)
    
    print('LAYER lines have been added successfully.')
else:
    print('No LAYER lines added; Vel2Grid3D has been choosen')

No LAYER lines added; Vel2Grid3D has been choosen


## Vel2Grid3D

*Given an existing 3D velocity model defined by interpolation between control point nodes and created by velocity inversion programs such as SimulPS, Simul2000 and FDTomo to a 3D model grid. (outputs a 3D Grid) , Vel2Grid3D generates a 3D model Grid header and buffer files containing velocities, slownesses or other model specification.*

This is the part used for SVI, using SimulPS_Vel2Grid notebook to create the files.

In [5]:
if which=='vel2grid3d':
    # Determine the filename for the SimulPS
    input_filename_simulPS = '/projects/amt/shared/lpapin/NLLoc/SVI/model/' # Location of the events
    
    # Read the original .run file
    with open(base_dir+'ttgrid_runfile_final.in', 'r') as file:
        lines = file.readlines()
    
    vginp_command = 'VGINP ' + input_filename_simulPS# + ' SIMUL2K ' ###not sure if something is missing
    # Find and replace the VGINP line in the .run file
    for i, line in enumerate(lines):
        if line.startswith('#VGINP'):
            lines[i] = vginp_command + ' #LP' + '\n'
            break
    
    # Write the modified content back to a new file (or overwrite the original)
    with open(base_dir+'ttgrid_runfile_final.in', 'w') as file:
        file.writelines(line.rstrip() + '\n' for line in lines)
    
    print('VGINP line has been modified successfully.')
else:
    print('No VGINP line added; Vel2Grid has been choosen')

VGINP line has been modified successfully.


## Grid2Time

*Given a velocity model grid, Grid2Time calculates the travel-time from a source point in a 3D grid to all other points in the grid. Optionally, the program also estimates the take-off angles for rays from each point in the grid to the source.*

The only line that needs to be modified is the one giving the coordinates of the stations that will be used for the locations. The INCLUDE line seems to be replacing the GTSOURCE lines.

In [6]:
# Determine the .txt file for station coordinates
input_filename_stas = base_dir + 'stations_coords_svi.txt'

include_command = 'INCLUDE ' + input_filename_stas
# Find and replace the INCLUDE line in the .run file
for i, line in enumerate(lines):
    if line.startswith('INCLUDE'):
        lines[i] = include_command + ' #LP' + '\n'
        break

# Write the modified content back to a new file (or overwrite the original)
with open(base_dir+'ttgrid_runfile_final.in', 'w') as file:
    file.writelines(line.rstrip() + '\n' for line in lines)

print('INCLUDE line has been added successfully.')

INCLUDE line has been added successfully.


## Runfile resulting

Just for clarity, the lines based on the type of Vel2Grid we are not using are removed. Then all lines for travel-time grid and location are merge into 1 runfile that will be use as input file for all the software.

In [7]:
# Determine the filename 
input_file = 'ttgrid_runfile_final.in'
with open(input_file, 'r') as infile:
    lines = infile.readlines()

# Keep the text without 1 of the block
modified_lines = []
inside_block = False
if which == 'vel2grid':
    for line in lines:
        if line.strip() == '## START -- Vel2Grid':
            inside_block = True  # Start removing lines
            continue
        if line.strip() == '## END -- Vel2Grid':
            inside_block = False  # Stop removing lines
            continue
        # Only keep the lines if we're not inside the block
        if not inside_block:
            modified_lines.append(line)
    with open(input_file, 'w') as outfile:
        outfile.writelines(modified_lines)
    print(f"Vel2Grid block removed from {input_file}.")
elif which == 'vel2grid3d':
    for line in lines:
        if line.strip() == '## START -- Vel2Grid3D':
            inside_block = True  # Start removing lines
            continue
        if line.strip() == '## END -- Vel2Grid3D':
            inside_block = False  # Stop removing lines
            continue
        # Only keep the lines if we're not inside the block
        if not inside_block:
            modified_lines.append(line)
    with open(input_file, 'w') as outfile:
        outfile.writelines(modified_lines)
    print(f"Vel2Grid3D block removed from {input_file}.")

Vel2Grid3D block removed from ttgrid_runfile_final.in.


In [8]:
# What does it look like after modifying the .run file for the commands
with open(base_dir+'ttgrid_runfile_final.in', 'r') as file:
    lines = file.readlines()
    non_comment_lines = [line.strip() for line in lines if not line.strip().startswith('#') and line.strip()]
    for line in non_comment_lines:
        print(line)

CONTROL 1 54321
TRANS SIMPLE 48.0 -124.5 0.0
VGOUT  ./model/layer
VGTYPE P
VGTYPE S
VGGRID 30 26 32 -124.5 48.0 0.0 0.37 0.55 3.0 VELOCITY #LP
GTFILES  ./model/layer  ./time/layer P
GTFILES  ./model/layer  ./time/layer S
GTMODE GRID2D ANGLES_YES
INCLUDE /Users/lpapin/Documents/phd/projects/sw4/nonlinloc/stations_coords_svi.txt #LP
GT_PLFD  1.0e-3  0


**Now the input file to use Vel2Grid and Grid2Time to create the travel times is ready.**

# 2. Location of events

## NLLoc

*NLLoc performs earthquake locations in 3D models using non-linear search techniques.*

I need to define the commands LOCFILES, LOCGRID, and TRANS SIMPLE. Not changing for now the LOCSEARCH command but it may be needed.

In [9]:
# Determine the filenames for LOCFILES
input_filename_obs = '/projects/amt/shared/lpapin/NLLoc/SVI/obs/' # Picks
input_filename_ttgrid = '/projects/amt/shared/lpapin/NLLoc/SVI/time/layer' # Travel-time grid
output_filename_loc = '/projects/amt/shared/lpapin/NLLoc/SVI/loc/' # Location of the events

# Read the original .run file
with open(base_dir+'nlloc_runfile_template.in', 'r') as file:
    lines = file.readlines()

locfiles_command = 'LOCFILES ' + input_filename_obs + ' NLL_OBS ' + input_filename_ttgrid + ' ' + output_filename_loc
# Find and replace the LOCFILES line in the .run file
for i, line in enumerate(lines):
    if line.startswith('LOCFILES'):
        lines[i] = locfiles_command + ' #LP' + '\n'
        break

# Write the modified content back to a new file (or overwrite the original)
with open(base_dir+'nlloc_runfile_final.in', 'w') as file:
    file.writelines(line.rstrip() + '\n' for line in lines)

print('LOCFILES line has been modified successfully.')

LOCFILES line has been modified successfully.


In [10]:
# Determine the command line for LOCGRID and replace
locgrid_command = 'LOCGRID ' + vggrid_command[7:-8] + 'PROB_DENSITY SAVE'
for i, line in enumerate(lines):
    if line.startswith('LOCGRID') and line.strip().endswith('PROB_DENSITY  SAVE'):
        lines[i] = locgrid_command + ' #LP' + '\n'
        break

with open(base_dir+'nlloc_runfile_final.in', 'w') as file:
    file.writelines(line.rstrip() + '\n' for line in lines)
    
print('LOCGRID line has been modified successfully.')

LOCGRID line has been modified successfully.


In [11]:
# Get the origins of the rectangle
parameters = vggrid_command.split()
latOrig = parameters[5]
longOrig = parameters[4]
rotAngle = 0.0

# Determine the command line for TRANS SIMPLE and replace
transsimple_command = 'TRANS SIMPLE ' + f'{latOrig} ' + f'{longOrig} ' + f'{rotAngle}' 
for i, line in enumerate(lines):
    if line.startswith('TRANS SIMPLE '):
        lines[i] = transsimple_command + ' #LP' + '\n'
        break

with open(base_dir+'nlloc_runfile_final.in', 'w') as file:
    file.writelines(line.rstrip() + '\n' for line in lines)

print('TRANS SIMPLE line has been modified successfully.')

TRANS SIMPLE line has been modified successfully.


## Runfile resulting

In [12]:
# What does it look like after modifying the .run file for the commands
with open(base_dir+'nlloc_runfile_final.in', 'r') as file:
    lines = file.readlines()
    non_comment_lines = [line.strip() for line in lines if not line.strip().startswith('#') and line.strip()]
    for line in non_comment_lines:
        print(line)

LOCSIG Lois Papin -- UO
LOCFILES /projects/amt/shared/lpapin/NLLoc/SVI/obs/ NLL_OBS /projects/amt/shared/lpapin/NLLoc/SVI/time/layer /projects/amt/shared/lpapin/NLLoc/SVI/loc/ #LP
LOCHYPOUT SAVE_NLLOC_ALL SAVE_HYPOINV_SUM
LOCSEARCH OCT 17 11 6 0.001 100000 10000 0 0
LOCMETH EDT_OT_WT 9999.0 4 -1 -1 -1 -1 -1 1
LOCGAU 0.2 0.0
LOCGAU2 0.01 0.05 2.0
LOCPHASEID  P   P
LOCPHASEID  S   S
LOCQUAL2ERR 0.1 0.5 1.0 2.0 99999.9
LOCGRID 30 26 32 -124.5 48.0 0.0 0.37 0.55 3.0 PROB_DENSITY SAVE #LP
LOCPHSTAT 9999.0 -1 9999.0 1.0 1.0 9999.9 -9999.9 9999.9
LOCANGLES ANGLES_YES 5
LOCMAG ML_HB 1.0 1.110 0.00189


# Merge all in one runfile

In [13]:
import glob
output_file = 'nlloc_svi.in'
input_files = glob.glob('*_final.in')[::-1]

In [14]:
# Open the output file for writing
with open(output_file, 'w') as outfile:
    # Iterate over all matching input files
    for fname in input_files:
        # Open each input file for reading
        with open(fname) as infile:
            # Write the contents of the input file to the output file
            outfile.write(infile.read())
            # Add a newline to separate file contents (optional)
            outfile.write('\n')

print(f"All input files merged into {output_file}")

All input files merged into nlloc_svi.in
